In [ ]:
pip install langchain-huggingface

In [9]:
%pip install -qU langchain-google-genai

In [10]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [8]:
!pip install langchain

In [6]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key:")

Enter your Google AI API key:··········


In [5]:
!pip install huggingface_hub

In [2]:
pip install -qU langchain-google-vertexai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 82.8 MB/s eta 0:00:00


In [3]:
%pip install -qU langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.3 MB/s eta 0:00:00


In [4]:
!pip install InstructorEmbedding


In [12]:
pip install --upgrade sentence-transformers transformers


In [ ]:
from google.colab import auth
auth.authenticate_user()


In [19]:
pip install -qU langchain-huggingface

In [26]:
pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.5 MB/s eta 0:00:00


In [34]:
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.docstore.document import Document

# Helper function to process grammar rules
def process_grammar_rules(data):
    chunks = []

    # Process verb conjugation rules
    for rule in data['sinhala_grammar_rules']['verb_conjugation_rules']:
        subject = rule['subject']
        for tense, details in rule['tense_endings'].items():
            chunks.append({
                'subject': subject,
                'tense': tense,
                'endings': details['endings'],
                'examples': details['examples'],
                'error_patterns': details['error_patterns']
            })

    # Process case markers
    for marker in data['sinhala_grammar_rules']['case_markers']:
        chunks.append({
            'case': marker['case'],
            'markers': marker['markers'],
            'examples': marker['examples'],
            'usage': marker['usage'],
            'error_patterns': marker['error_patterns']
        })

    # Process error detection patterns
    for error_pattern in data['sinhala_grammar_rules']['error_detection_patterns']:
        chunks.append({
            'error_type': error_pattern['error_type'],
            'pattern': error_pattern['pattern'],
            'examples': error_pattern['examples']
        })

    # Process temporal markers
    for temporal_marker in data['sinhala_grammar_rules']['temporal_markers'].values():
        for marker in temporal_marker:
            chunks.append({'temporal_marker': marker})

    return chunks

# Load Sinhala grammar rules data
with open('/content/sinhala_grammar.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Preprocess grammar rules
chunks = process_grammar_rules(data)

# Split chunks into smaller parts for indexing
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
formatted_chunks = [
    Document(page_content=json.dumps(chunk, ensure_ascii=False))
    for chunk in chunks
    for chunk in text_splitter.split_text(json.dumps(chunk, ensure_ascii=False))
]

# Initialize embeddings model
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vector_store = FAISS.from_documents(formatted_chunks, embeddings_model)

# Initialize LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

# Define prompt template
prompt_template = ChatPromptTemplate.from_template("""
<context>
{context}
</context>

Question: {input}

Please provide a corrected sentence and explain the errors.
""")

# Create document and retrieval chains
document_chain = create_stuff_documents_chain(llm, prompt_template)
retriever = vector_store.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# Context and query for Sinhala Grammar Error Checking
context = """Input:
You will receive a Sinhala sentence. It may contain grammatical errors, spelling mistakes, or both.

Output:
- Highlight errors in spelling or grammar.
- Explain the errors with examples.
- Provide the corrected sentence.

For example:
Input Sentence: "මම කෑම ගනිමු"
Output:
Errors Identified:
Grammar Error: Verb "ගනිමු" does not match the subject "මම."
Explanation: The subject "මම" requires the verb to end with "මි." Correct verb: "ගනිමි."
Corrected Sentence: "මම කෑම ගනිමි"
"""
query = "ඔහු ගෙදර ගියෙමි"

# Get response
response = retrieval_chain.invoke({"context": context, "input": query})

# Display the response
print("Response:")
print(response["answer"])


Response:
**Corrected Sentence:** ඔහු ගෙදර ගියේය.

**Explanation of Errors:**

The original sentence, "ඔහු ගෙදර ගියෙමි," contains an error in the verb ending. Let's break it down:

*   **ඔහු (ohu):** This is the subject pronoun, meaning "he."
*   **ගෙදර (gedara):** This means "home" and is the location.
*   **ගියෙමි (giyemi):** This is the verb "to go" in the past tense, but the ending "මි" is incorrect for the subject "ඔහු".

**The Error:**

The verb ending "මි" is used when the subject is "මම" (I) in the present tense. The provided context shows that "මම" takes the endings "මි" or "ෙමි" in the present tense. However, the sentence is in the past tense, and the subject is "ඔහු" (he).

**The Correction:**

The correct past tense ending for "ඔහු" (he) is "ය" or "ඒය". Therefore, the correct verb form is "ගියේය" (giyeeya).

**In summary:** The error was using the first-person singular present tense verb ending "මි" with the third-person singular subject "ඔහු" in the past tense. The correct